In [2]:
from config import *
from LoadDataset import CustomImageDataset
from torch.utils.data import DataLoader
from torchvision import transforms
from config import dataset_test_path, dataset_train_path, dataset_val_path

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CustomImageDataset(file_path=dataset_train_path, folder_path=dataset_root, transform=transform)
val_dataset = CustomImageDataset(file_path=dataset_val_path, folder_path=dataset_root, transform=transform)
test_dataset = CustomImageDataset(file_path=dataset_test_path, folder_path=dataset_root, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [3]:
len(train_dataset), len(val_dataset), len(test_dataset), "total:", len(train_dataset) + len(val_dataset) + len(test_dataset)

(674373, 207499, 155625, 'total:', 1037497)

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import models, transforms
import time
import copy
from torch.utils.data import DataLoader

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = CustomImageDataset(file_path=dataset_train_path, folder_path=dataset_root, transform=transform)
val_dataset = CustomImageDataset(file_path=dataset_val_path, folder_path=dataset_root, transform=transform)
test_dataset = CustomImageDataset(file_path=dataset_test_path, folder_path=dataset_root, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pre-trained ResNet50 model with the new weights parameter
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)

# Modify the final fully connected layer to match the number of classes in the dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

# Move the model to the device (GPU or CPU)
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

# Define the learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

# Training function
def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels, _ in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

# Prepare dataloaders dictionary
dataloaders = {
    'train': train_loader,
    'val': val_loader
}

Using device: cpu


In [9]:

# Train the model
model = train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 0/14
----------


KeyboardInterrupt: 

In [ ]:

# Save the trained model
torch.save(model.state_dict(), f'{dataset_root}resnet50_clothing1m.pth')
print("Model saved.")

In [8]:
import torch
from torchvision import models
from torchviz import make_dot

# Define the model (assuming you have already defined and loaded it as before)
weights = models.ResNet50_Weights.IMAGENET1K_V1
model = models.resnet50(weights=weights)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

# Create a dummy input tensor with the same shape as your input data
dummy_input = torch.randn(1, 3, 224, 224).to(device)

# Perform a forward pass using the dummy input
output = model(dummy_input)

# Visualize the model
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = 'png'
dot.render('model/resnet50_model')

# Display the model graph
from IPython.display import Image
Image('model/resnet50_model.png')